In [1]:
import sys, os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import (
    top_k_accuracy_score,
    classification_report,
    confusion_matrix
)
import random
from pathlib import Path

In [2]:
# Compute absolute path to the `src/` folder
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH     = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

from utils import get_dataloaders, get_dataloaders_continents, load_model, evaluate_model, print_metrics, plot_confusion_matrix, show_sample_predictions

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Using device:", device)

Using device: cuda


# Countries

In [ ]:
COUNTRIES = ['Albania', 'Andorra', 'Australia', 'Austria', 'Bangladesh', 'Belgium', 'Bhutan', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Chile', 'Colombia', 'Croatia', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Estonia', 'Eswatini', 'Finland', 'France', 'Germany', 'Greece', 'Guatemala', 'Hungary', 'Iceland', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Latvia', 'Lesotho', 'Lithuania', 'Luxembourg', 'Malaysia', 'Mexico', 'Montenegro', 'Netherlands', 'New Zealand', 'North Macedonia', 'Norway', 'Palestine', 'Peru', 'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'United Arab Emirates', 'United Kingdom', 'United States']
num_classes = len(COUNTRIES)
project_root   = Path().resolve().parent

### Data

In [5]:
segments = ['road', 'vegetation', 'terrain']

In [6]:
for i in segments:
    res = ""
    test_root = project_root / "datasets" / "segmented" / i / "final_datasets" / "test"
    test_loader = get_dataloaders(test_root, batch_size=32)
    model_dir = project_root / "models" / f"resnet_finetuned_{i}"
    model = load_model(model_path=model_dir / "main.pth", device=device, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()
    avg_loss, top1_acc, all_targets, all_preds, all_probs = evaluate_model(model, test_loader, criterion, device)
    
    with open(f"{model_dir}/eval.txt", "a") as f:
        f.write(f"\nResults for {i}:\n")
        f.write(f"Average loss: {avg_loss:.4f}\n")
        f.write(f"Top-1 accuracy: {top1_acc:.4f}\n")
        f.write("\n")
        top3, top5, report = print_metrics(all_targets, all_preds, all_probs, COUNTRIES)
        f.write(f"Top-3 accuracy: {top3:.4f}\n")
        f.write(f"Top-5 accuracy: {top5:.4f}\n")
        f.write("\nClassification Report:\n")
        f.write(report)
        f.write("\n" + "="*50 + "\n")

/home/andreafabbricatore/rainbot/src/utils.py:249: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
/home/and

# Continents

In [16]:
CONTINENTS = ["Africa", "Asia", "Europe", "North America", "Oceania", "South America"]
num_classes = len(CONTINENTS)
project_root   = Path().resolve().parent
# Find all model folders containing "continents"
model_folders = list(project_root.glob("models/*continents*"))

# Extract the segment name (word before _continents) from each folder
segments = ["_".join(folder.name.split("_")[2:-1]) for folder in model_folders]

print("Found segments:", segments)


Found segments: ['fence', 'traffic_sign', 'terrain', 'traffic_light', 'bicycle', 'car', 'sidewalk', 'building', 'vegetation', 'pole', 'person', 'road']


### Data

In [17]:
for i in segments:
    res = ""
    test_root = project_root / "datasets" / "segmented" / i / "final_datasets_continents" / "test"
    test_loader = get_dataloaders_continents(test_root, batch_size=32)
    model_dir = project_root / "models" / f"resnet_finetuned_{i}_continents"
    model = load_model(model_path=model_dir / "main.pth", device=device, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()
    avg_loss, top1_acc, all_targets, all_preds, all_probs = evaluate_model(model, test_loader, criterion, device)
    
    with open(f"{model_dir}/eval.txt", "a") as f:
        f.write(f"\nResults for {i}:\n")
        f.write(f"Average loss: {avg_loss:.4f}\n")
        f.write(f"Top-1 accuracy: {top1_acc:.4f}\n")
        f.write("\n")
        top3, top5, report = print_metrics(all_targets, all_preds, all_probs, CONTINENTS)
        f.write(f"Top-3 accuracy: {top3:.4f}\n")
        f.write(f"Top-5 accuracy: {top5:.4f}\n")
        f.write("\nClassification Report:\n")
        f.write(report)
        f.write("\n" + "="*50 + "\n")

/home/andreafabbricatore/rainbot/src/utils.py:249: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
/home/and